In [19]:
from textgrid import TextGrid
import numpy as np
import os

pwd = os.getcwd()

tg_en_pwd = os.getcwd() + "/sound_sample/AI_eng_yunjang/mine_en.TextGrid"
tg_ko_pwd = os.getcwd() + "/sound_sample/korean_yunjang/윤장목소리1.TextGrid"

# 1. TextGrid 로드
tg_ko = TextGrid.fromFile(tg_ko_pwd)
tg_en = TextGrid.fromFile(tg_en_pwd)

tier_ko = tg_ko.getFirst("words")
tier_en = tg_en.getFirst("words")

# 2. 매핑 정의 (수동)

sentence_mapping = [
    (["안녕", "하", "세요"], ["hello"]),
    (["제", "이름", "은", "조윤장", "입니다"], ["my", "name", "is", "jo", "yoon", "jang"]),
    (["만나", "서", "반갑", "습니다"], ["nice", "to", "meet", "you"]),
    (["잘", "부탁", "드립니다"], ["please", "take", "care", "of", "me"])
]

def normalize(text):
    return text.replace(" ", "").replace(".", "").replace(",", "").lower()

def find_combined_interval_multiple(tier, word_list):
    """
    단어 리스트 중 모든 단어가 순서대로 포함되면 해당 구간 반환
    """
    normalized_words = [normalize(w) for w in word_list if w.strip()]
    match_indices = []
    
    for i in range(len(tier)):
        mark = normalize(tier[i].mark)
        if not mark:
            continue

        # 현재 interval부터 단어 리스트 순서대로 매칭 가능한지 확인
        matched = True
        j = 0
        k = i
        while j < len(normalized_words) and k < len(tier):
            mark_k = normalize(tier[k].mark)
            if mark_k == "":
                k += 1
                continue
            if normalized_words[j] != mark_k:
                matched = False
                break
            j += 1
            k += 1

        if matched:
            return (tier[i].minTime, tier[k - 1].maxTime)

    return None

# 3. Isochrony 측정
duration_diffs = []

for ko_words, en_words in sentence_mapping:
    ko_range = find_combined_interval_multiple(tier_ko, ko_words)
    en_range = find_combined_interval_multiple(tier_en, en_words)

    if ko_range is None or en_range is None:
        print(f"⚠️ 누락된 매핑: {' '.join(ko_words)} ↔ {' '.join(en_words)}")
        continue

    ko_dur = ko_range[1] - ko_range[0]
    en_dur = en_range[1] - en_range[0]
    duration_diffs.append(abs(ko_dur - en_dur))


# 4. 정규화 점수 계산
if duration_diffs:
    total_duration = max(tier_ko.maxTime, tier_en.maxTime)
    ichron_score = 1 - (np.mean(duration_diffs) / total_duration)
    print(f"\n✅ Isochrony Score: {ichron_score:.3f}")
else:
    print("매핑된 문장이 없어서 점수를 계산할 수 없습니다.")


✅ Isochrony Score: 0.929
